In [1]:
import pandas as pd

def get_new_feature_name_df(old_feature_name_df):
  feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
  feature_dup_df = feature_dup_df.reset_index()
  
  new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
  new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                             if x[1] > 0 else x[0], axis=1)
  new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
  return new_feature_name_df

def get_human_dataset():
  #feature.txt 파일에는 피처 이름 index와 ㅍ첨ㅇ이 공백으로 분리되어 있음, 이를 DataFrame으로 로드  
  feature_name_df = pd.read_csv('../../../예제데이터/human_activity/features.txt', sep='\s+', header=None, names=['column_index','column_name'])
  
  # 중복된 피처명을 수정하는 get_enw_feature_name_df() 를 이용 
  new_feature_name_df = get_new_feature_name_df(feature_name_df)
  
  # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환 
  feature_name = new_feature_name_df.iloc[:,1].values.tolist()
  
  # 학습 피처 데이터세트와 테스트피처 데이터를 DataFrame으로 로딩. 칼럼명은 feature_name 적용 
  X_train = pd.read_csv('../../../예제데이터/human_activity/train/X_train.txt', sep='\s+', names=feature_name)
  X_test = pd.read_csv('../../../예제데이터/human_activity/test/X_test.txt', sep='\s+', names=feature_name)
  
  # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩. 칼럼명은 action으로 적용 
  y_train = pd.read_csv('../../../예제데이터/human_activity/train/y_train.txt', sep='\s+', header=None, names=['action'])
  y_test = pd.read_csv('../../../예제데이터/human_activity/test/y_test.txt', sep='\s+', header=None, names=['action'])
  
  # 로드된 하급/테스트용 DataFrame을 모두 반환
  return X_train, X_test, y_train, y_test

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

#GBM 수행 시간 측정을 위함. 시작 시간 설정

start_time = time.time()

## 약한 학습기의 순차적인 예측 오류 버정을 통해 학습을 수행하므로 멀티 CPU 코어 시스템을 사용하더라도 병렬처리가 지원되지 않음
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print("GBM 정확도: {0:.4f}".format(gb_accuracy))
print("GBM 수행 시간: {0:.1f} 초".format(time.time()-start_time))

GBM 정확도: 0.9389
GBM 수행 시간: 436.8 초


In [6]:
from sklearn.model_selection import GridSearchCV

params = {
  'n_estimators':[100, 500],
  'learning_rate':[0.05, 0.1]  
}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터: \n', grid_cv.best_params_)
print('최고 예측 정확도:{0:.4}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 4 candidates, totalling 8 fits


KeyboardInterrupt: 